In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
import os
# Make sure we're in the right directory
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

In [2]:
print('Hi :)')

Hi :)


In [12]:
import sys
import h5py
import json
import logging
import torch
from torch import Tensor
import numpy as np
from einops import repeat, rearrange
import time
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict
import rtml.data_wrangling.constants as constants
from rtml.models.interface import get_model, is_gnn, is_graph_net
from rtml.models.column_handler import ColumnPreprocesser
from rtml.models.GNs.constants import NODES, EDGES
from rtml.data_wrangling.constants import LEVELS, LAYERS, GLOBALS, TRAIN_YEARS, TEST_YEARS

In [13]:
torch.cuda.get_device_name(0)

'Tesla V100-SXM2-32GB'

In [14]:
plt.rcParams['figure.figsize'] = [20, 8]  # general matplotlib parameters
plt.rcParams['figure.dpi'] = 70 
np.set_printoptions(suppress=True, threshold=sys.maxsize)

In [15]:
hdf5_years_dir = "/miniscratch/salva.ruhling-cachay/ECC_data/snapshots/1979-2014/hdf5/inputs"
year = 2011
h5_path = os.path.join(hdf5_years_dir, str(year) + '.h5')

In [16]:
def get_one_snapshot(h5_path: str, batch_size: int, exp_type= 'pristine'):
    n_batches = int(8192/batch_size)
    data = []
    with h5py.File(h5_path, 'r') as h5f:
        globs = np.array(h5f[GLOBALS])
        lays = np.array(h5f[LAYERS][..., :14]) if exp_type == 'pristine' else np.array(h5f[LAYERS])
        levs = np.array(h5f[LEVELS])
        for i in range(n_batches):
            dslice = slice(i*batch_size, (i+1)*batch_size)
            data.append(
                {GLOBALS: globs[dslice], LAYERS: lays[dslice], LEVELS: levs[dslice]}
            )
    del globs, lays, levs
    print(f"{n_batches} batches of size {batch_size}, amounting to {sum([d[GLOBALS].shape[0] for d in data])} data points.")
    return data


In [17]:
def gn_input_dict_renamer_level_nodes(batch: Dict[str, Tensor], device):
    batch[NODES] = torch.FloatTensor(batch[LEVELS]).to(device)
    batch[GLOBALS] = torch.FloatTensor(batch[GLOBALS]).to(device)
    batch[EDGES] = torch.FloatTensor(batch[LAYERS])
    batch[EDGES] = repeat(batch[EDGES], "b e d -> b (repeat e) d", repeat=2).to(device)  # bidirectional edges
    return batch

def mlp_transform(batch: Dict[str, Tensor], device):
    return torch.cat([torch.FloatTensor(v).reshape((v.shape[0], -1)).to(device) for v in batch.values()], dim=1)

def to_torch(batch, device):
    return {k: torch.FloatTensor(v).to(device) for k, v in batch.items()}

def to_torch2(batch, device):
    return {
            LEVELS: torch.FloatTensor(batch[LEVELS]).to(device),
            GLOBALS: torch.FloatTensor(batch[GLOBALS]).to(device),
            LAYERS: torch.FloatTensor(batch[LAYERS]).to(device)
    }
def cnn_transform(batch, device):
    X_levels = torch.FloatTensor(batch[LEVELS])

    X_layers = rearrange(torch.nn.functional.pad(rearrange(torch.FloatTensor(batch[LAYERS]), 'b c f -> ()b c f'), (0,0,1,0),\
            mode='reflect'), '()b c f ->b c f')
    X_global = repeat(torch.FloatTensor(batch[GLOBALS]), 'b f ->b c f', c = 50)

    X = torch.cat((X_levels, X_layers, X_global), -1)
    return rearrange(X, 'b c f -> b f c').to(device)

in_transform_funcs = {
    "mlp": mlp_transform,
    'gn': gn_input_dict_renamer_level_nodes, 
    'gcn': to_torch2,
    'cnn': cnn_transform
}

In [18]:
def reload_and_speed_test_model(ckpt: str, data, device, model_dir='scripts/out', init_runs = 2, avg_runs_over = 10):
    """ init_runs are run but not accounted for in the benchmark, i.e. warm-up runs. """
    model_ckpt = torch.load(f"{model_dir}/{ckpt}.pkl", map_location=torch.device(device))
    params = model_ckpt['hyper_params']
    net_params = model_ckpt['model_params']
    model_type = params['model']
    
    model_kwargs = dict()
    if is_gnn(model_type) or is_graph_net(model_type):
        model_kwargs['column_preprocesser'] = ColumnPreprocesser(
            n_layers=49, input_dims=net_params['input_dim'], **params['preprocessing_dict']
        )
    model = get_model(model_name=model_type, device=device, verbose=False, **model_ckpt['model_params'], **model_kwargs)
    model.load_state_dict(model_ckpt['model'])
    model = model.to(device).float()
    model.eval()
    
    transform = in_transform_funcs[model_type.lower().replace('+readout', '')]
    
    loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
    for logger in loggers:
        logger.setLevel(logging.WARNING)
    
    times = []
    total_times = []

    for it in range(init_runs + avg_runs_over):
        batch_time = batch_total_time = 0
        for i, batch in enumerate(data, 1):
            batch = batch.copy()
            total_start = torch.cuda.Event(enable_timing=True)
            start_t = torch.cuda.Event(enable_timing=True)
            end_t = torch.cuda.Event(enable_timing=True)
            total_start.record()
            batch_model = transform(batch, device=device)
            start_t.record()
            with torch.no_grad():
                Y = model(batch_model)
            end_t.record()
            
            torch.cuda.synchronize()
            
            forward_t = start_t.elapsed_time(end_t)/1000
            total_t = total_start.elapsed_time(end_t)/1000
            batch_time += forward_t
            batch_total_time += total_t
        print(f"Forward pass needed {batch_time} out of {batch_total_time} sec. for {i} batches.")
        if it > init_runs:
            times += [batch_time]
            total_times += [batch_total_time]

    times, total_times = np.array(times), np.array(total_times)
    mean1, std1 = times.mean(), times.std()
    mean2, std2 = total_times.mean(), total_times.std()

    print(f"Forward time for {model_type} on {device}: ${mean1:.4f} \pm {std1:.3f}$, Total time with reshaping: ${mean2:.4f} \pm {std2:.3f}$ (batch-size={batch_size})")

In [19]:
# model_dir1 = "/home/mila/s/salva.ruhling-cachay/RT-DL/scripts/out"
model_dir1 = "/home/mila/v/venkatesh.ramesh/rt_ml/main_branch/test/Radiative_transfer_dl/scripts2/out"
model_dir2 = "scripts/out"

## Speed test CPU + batch size = 512

In [52]:
batch_size = 512
data = get_one_snapshot(h5_path, batch_size, exp_type='pristine')

16 batches of size 512, amounting to 8192 data points.


In [26]:
mlp_model_ckpt = "0.1115valMAE_199ep_MLP_1990+1999+2003train_2005val_Z_7seed_11h39m_on_Aug_25_2dajv2lx"
gn_model_ckpt = "0.1763valMAE_94ep_GN+READOUT_1990+1999+2003train_2005val_Z_99seed_02h12m_on_Aug_25_j9nsvo7s"
gcn_model_ckpt = "0.4532valMAE_138ep_GCN+READOUT_1990+1999+2003train_2005val_Z_7seed_13h18m_on_Aug_23_1xk39ugw"
cnn_model_ckpt = "0.0922valMAE_100ep_CNN_1990+1999+2003train_2005val_Z_7seed_02h28m_on_Aug_26_1d9egkrs"

In [ ]:
reload_and_speed_test_model(mlp_model_ckpt, data, model_dir=model_dir1, device = 'cpu')

In [ ]:
reload_and_speed_test_model(gn_model_ckpt, data, model_dir=model_dir1, device = 'cpu')

In [107]:
reload_and_speed_test_model(gcn_model_ckpt, data, model_dir=model_dir1, device = 'cpu')

INFO:globals_MLP_projector: No inverse normalization for outputs is used.
INFO:levels_MLP_projector: No inverse normalization for outputs is used.
INFO:layers_MLP_projector: No inverse normalization for outputs is used.
INFO:GCN_Readout_MLP: No inverse normalization for outputs is used.


Forward pass needed 8.324446319580078 out of 8.32660479736328 sec. for 16 batches.
Forward pass needed 8.321710021972656 out of 8.323666839599609 sec. for 16 batches.
Forward pass needed 8.376828918457031 out of 8.378868774414062 sec. for 16 batches.
Forward pass needed 8.530550842285155 out of 8.53254748535156 sec. for 16 batches.
Forward pass needed 8.588445739746094 out of 8.590463012695313 sec. for 16 batches.
Forward pass needed 8.627592346191406 out of 8.629627929687498 sec. for 16 batches.
Forward pass needed 8.481688720703124 out of 8.483694641113281 sec. for 16 batches.
Forward pass needed 8.509514770507813 out of 8.511604736328126 sec. for 16 batches.
Forward pass needed 8.492995605468751 out of 8.495072204589842 sec. for 16 batches.
Forward pass needed 8.506913757324218 out of 8.508918823242187 sec. for 16 batches.
Forward pass needed 8.595684326171876 out of 8.597728271484375 sec. for 16 batches.
Forward pass needed 8.507639831542969 out of 8.509634582519531 sec. for 16 bat

In [ ]:
reload_and_speed_test_model(cnn_model_ckpt, data, model_dir=model_dir2, device = 'cpu')

# Pristine, GPU, 512 batch-size

In [ ]:
reload_and_speed_test_model(mlp_model_ckpt, data, device = 'cuda')

In [76]:
reload_and_speed_test_model(gn_model_ckpt, data, model_dir=model_dir1, device = 'cuda')

INFO:GN_readout_MLP: No inverse normalization for outputs is used.


_------------------------- True
Forward pass needed 3.5268044738769526 out of 3.544875119209289 sec. for 16 batches.
Forward pass needed 0.21128582382202146 out of 0.23019414138793948 sec. for 16 batches.
Forward pass needed 0.1954006071090698 out of 0.21209398365020754 sec. for 16 batches.
Forward pass needed 0.19211660861969 out of 0.20890521812438964 sec. for 16 batches.
Forward pass needed 0.1913344955444336 out of 0.20809833717346188 sec. for 16 batches.
Forward pass needed 0.19368988990783692 out of 0.211979266166687 sec. for 16 batches.
Forward pass needed 0.1910831346511841 out of 0.20786380767822263 sec. for 16 batches.
Forward pass needed 0.19085414409637452 out of 0.20764262294769287 sec. for 16 batches.
Forward pass needed 0.1915508804321289 out of 0.20820479869842523 sec. for 16 batches.
Forward pass needed 0.1938556776046753 out of 0.21040230464935303 sec. for 16 batches.
Forward pass needed 0.19059436702728272 out of 0.20732924842834471 sec. for 16 batches.
Forward pass 

In [100]:
reload_and_speed_test_model(gcn_model_ckpt, data, model_dir=model_dir1, device = 'cuda')

INFO:globals_MLP_projector: No inverse normalization for outputs is used.
INFO:levels_MLP_projector: No inverse normalization for outputs is used.
INFO:layers_MLP_projector: No inverse normalization for outputs is used.
INFO:GCN_Readout_MLP: No inverse normalization for outputs is used.


Forward pass needed 2.1776693878173825 out of 2.190135307312011 sec. for 16 batches.
Forward pass needed 2.0077733459472658 out of 2.0186623992919923 sec. for 16 batches.
Forward pass needed 2.011904182434082 out of 2.022813705444336 sec. for 16 batches.
Forward pass needed 2.011231727600098 out of 2.022241271972656 sec. for 16 batches.
Forward pass needed 2.0112457733154296 out of 2.022254585266113 sec. for 16 batches.
Forward pass needed 2.0136365661621096 out of 2.024816650390625 sec. for 16 batches.
Forward pass needed 2.043749099731445 out of 2.0566937713623044 sec. for 16 batches.
Forward pass needed 2.015239624023437 out of 2.026893302917481 sec. for 16 batches.
Forward pass needed 2.0128231201171873 out of 2.023723014831543 sec. for 16 batches.
Forward pass needed 2.00696110534668 out of 2.0178728866577154 sec. for 16 batches.
Forward pass needed 2.007321876525879 out of 2.0183091278076173 sec. for 16 batches.
Forward pass needed 2.0186029052734376 out of 2.030065673828125 sec.

# Pristine, GPU, 2048 batch-size

In [23]:
batch_size = 2048
data_b2048 = get_one_snapshot(h5_path, batch_size, exp_type='pristine')

4 batches of size 2048, amounting to 8192 data points.


In [95]:
reload_and_speed_test_model(gn_model_ckpt, data_b2048, model_dir=model_dir1, device = 'cuda')

INFO:GN_readout_MLP: No inverse normalization for outputs is used.


Forward pass needed 0.22000089645385743 out of 0.23112499237060546 sec. for 4 batches.
Forward pass needed 0.1964249610900879 out of 0.20764262771606445 sec. for 4 batches.
Forward pass needed 0.18285052871704102 out of 0.1946019821166992 sec. for 4 batches.
Forward pass needed 0.17778924560546874 out of 0.1896857566833496 sec. for 4 batches.
Forward pass needed 0.1772358055114746 out of 0.18816614151000977 sec. for 4 batches.
Forward pass needed 0.17634259414672854 out of 0.18727324676513674 sec. for 4 batches.
Forward pass needed 0.17842928314208983 out of 0.18973798370361328 sec. for 4 batches.
Forward pass needed 0.17914271926879882 out of 0.19078453826904296 sec. for 4 batches.
Forward pass needed 0.17853923416137696 out of 0.19036877059936524 sec. for 4 batches.
Forward pass needed 0.17773980712890627 out of 0.1892075538635254 sec. for 4 batches.
Forward pass needed 0.17903062438964842 out of 0.190166015625 sec. for 4 batches.
Forward pass needed 0.17856844711303713 out of 0.1893

In [98]:
reload_and_speed_test_model(gcn_model_ckpt, data_b2048, model_dir=model_dir1, device = 'cuda')

INFO:globals_MLP_projector: No inverse normalization for outputs is used.
INFO:levels_MLP_projector: No inverse normalization for outputs is used.
INFO:layers_MLP_projector: No inverse normalization for outputs is used.
INFO:GCN_Readout_MLP: No inverse normalization for outputs is used.


Forward pass needed 2.429603942871094 out of 2.435315673828125 sec. for 4 batches.
Forward pass needed 2.2146614990234372 out of 2.2208184814453125 sec. for 4 batches.
Forward pass needed 2.214556945800781 out of 2.2208850097656248 sec. for 4 batches.
Forward pass needed 2.217142944335938 out of 2.2232023315429688 sec. for 4 batches.
Forward pass needed 2.2139124145507814 out of 2.2197012939453122 sec. for 4 batches.
Forward pass needed 2.2110499267578128 out of 2.216841186523437 sec. for 4 batches.
Forward pass needed 2.2489202270507813 out of 2.2552760009765622 sec. for 4 batches.
Forward pass needed 2.215150329589844 out of 2.2217625732421875 sec. for 4 batches.
Forward pass needed 2.213172119140625 out of 2.2195444946289062 sec. for 4 batches.
Forward pass needed 2.2132908935546878 out of 2.219939819335938 sec. for 4 batches.
Forward pass needed 2.229229736328125 out of 2.2358701782226564 sec. for 4 batches.
Forward pass needed 2.2095022583007813 out of 2.2153707275390624 sec. for 

In [29]:
reload_and_speed_test_model(cnn_model_ckpt, data_b2048, model_dir=model_dir1, device = 'cuda')

Forward pass needed 0.01688489627838135 out of 0.1570764808654785 sec. for 4 batches.
Forward pass needed 0.017217887878417968 out of 0.15595417785644533 sec. for 4 batches.
Forward pass needed 0.017074079990386962 out of 0.15110553359985351 sec. for 4 batches.
Forward pass needed 0.01698102378845215 out of 0.1521530876159668 sec. for 4 batches.
Forward pass needed 0.01702668809890747 out of 0.1574676475524902 sec. for 4 batches.
Forward pass needed 0.01701107215881348 out of 0.1590118408203125 sec. for 4 batches.
Forward pass needed 0.017023072242736817 out of 0.17266381072998047 sec. for 4 batches.
Forward pass needed 0.01707263994216919 out of 0.16868659210205078 sec. for 4 batches.
Forward pass needed 0.017040223598480224 out of 0.17019494628906248 sec. for 4 batches.
Forward pass needed 0.016937184333801272 out of 0.1638901786804199 sec. for 4 batches.
Forward pass needed 0.017051104068756102 out of 0.16162303924560545 sec. for 4 batches.
Forward pass needed 0.017090943813323975 o

# Pristine, GPU, 8192 batch-size

In [24]:
batch_size = 8192
data_b8192 = get_one_snapshot(h5_path, batch_size, exp_type='pristine')

1 batches of size 8192, amounting to 8192 data points.


In [105]:
reload_and_speed_test_model(gn_model_ckpt, data_b8192, model_dir=model_dir1, device = 'cuda')

INFO:GN_readout_MLP: No inverse normalization for outputs is used.


Forward pass needed 0.2125672607421875 out of 0.23115676879882813 sec. for 1 batches.
Forward pass needed 0.1933829803466797 out of 0.21477682495117187 sec. for 1 batches.
Forward pass needed 0.1849432373046875 out of 0.20714802551269532 sec. for 1 batches.
Forward pass needed 0.1817329864501953 out of 0.20408934020996095 sec. for 1 batches.
Forward pass needed 0.18316117858886719 out of 0.2052894744873047 sec. for 1 batches.
Forward pass needed 0.18484162902832033 out of 0.20613836669921876 sec. for 1 batches.
Forward pass needed 0.18573164367675782 out of 0.20717669677734374 sec. for 1 batches.
Forward pass needed 0.18459292602539062 out of 0.20603904724121094 sec. for 1 batches.
Forward pass needed 0.18115005493164063 out of 0.2026956787109375 sec. for 1 batches.
Forward pass needed 0.18330624389648437 out of 0.20548095703125 sec. for 1 batches.
Forward pass needed 0.18474214172363282 out of 0.206993408203125 sec. for 1 batches.
Forward pass needed 0.18583287048339844 out of 0.20869

In [104]:
reload_and_speed_test_model(gcn_model_ckpt, data_b8192, model_dir=model_dir1, device = 'cuda')

INFO:globals_MLP_projector: No inverse normalization for outputs is used.
INFO:levels_MLP_projector: No inverse normalization for outputs is used.
INFO:layers_MLP_projector: No inverse normalization for outputs is used.
INFO:GCN_Readout_MLP: No inverse normalization for outputs is used.


Forward pass needed 2.500475341796875 out of 2.505830322265625 sec. for 1 batches.
Forward pass needed 2.4892607421875 out of 2.4941015625 sec. for 1 batches.
Forward pass needed 2.495714599609375 out of 2.500497314453125 sec. for 1 batches.
Forward pass needed 2.49080517578125 out of 2.49580224609375 sec. for 1 batches.
Forward pass needed 2.487874267578125 out of 2.49291259765625 sec. for 1 batches.
Forward pass needed 2.492655029296875 out of 2.49736181640625 sec. for 1 batches.
Forward pass needed 2.49445556640625 out of 2.499492919921875 sec. for 1 batches.
Forward pass needed 2.486072998046875 out of 2.490841064453125 sec. for 1 batches.
Forward pass needed 2.489851806640625 out of 2.49501806640625 sec. for 1 batches.
Forward pass needed 2.4933662109375 out of 2.498262939453125 sec. for 1 batches.
Forward pass needed 2.501635009765625 out of 2.505985107421875 sec. for 1 batches.
Forward pass needed 2.489495849609375 out of 2.49444873046875 sec. for 1 batches.
Forward time for GCN

In [30]:
reload_and_speed_test_model(cnn_model_ckpt, data_b8192, model_dir=model_dir1, device = 'cuda')

Forward pass needed 0.015430399894714356 out of 0.16836300659179687 sec. for 1 batches.
Forward pass needed 0.015467328071594238 out of 0.15744306945800782 sec. for 1 batches.
Forward pass needed 0.015439264297485352 out of 0.158350341796875 sec. for 1 batches.
Forward pass needed 0.015372672080993653 out of 0.15237017822265625 sec. for 1 batches.
Forward pass needed 0.015432640075683594 out of 0.15401983642578124 sec. for 1 batches.
Forward pass needed 0.015388256072998046 out of 0.15064576721191406 sec. for 1 batches.
Forward pass needed 0.015411392211914062 out of 0.1504102325439453 sec. for 1 batches.
Forward pass needed 0.015462719917297363 out of 0.1493053436279297 sec. for 1 batches.
Forward pass needed 0.015371328353881835 out of 0.14976921081542968 sec. for 1 batches.
Forward pass needed 0.015440159797668458 out of 0.14833970642089844 sec. for 1 batches.
Forward pass needed 0.01538742446899414 out of 0.15044403076171875 sec. for 1 batches.
Forward pass needed 0.015430720329284

# Clear-sky

In [80]:
gn_model_ckpt_cs = "0.3491valMAE_97ep_GN+READOUT_CS_1990+1999+2003train_2005val_Z_7seed_12h58m_on_Aug_25_3emesh6i"
gcn_model_ckpt_cs = "0.5462valMAE_189ep_GCN+READOUT_CS_1990+1999+2003train_2005val_Z_7seed_03h09m_on_Aug_23_1gdj0tl7"

In [83]:
batch_size = 512
data_cs = get_one_snapshot(h5_path, batch_size, exp_type='clear_sky')

16 batches of size 512, amounting to 8192 data points.


In [84]:
reload_and_speed_test_model(gn_model_ckpt_cs, data_cs, model_dir=model_dir1, device = 'cuda')

INFO:GN_readout_MLP: No inverse normalization for outputs is used.


_------------------------- True
Forward pass needed 0.23414691162109375 out of 0.2700646438598633 sec. for 16 batches.
Forward pass needed 0.20768921661376955 out of 0.23975526523590088 sec. for 16 batches.
Forward pass needed 0.19524076747894287 out of 0.23038259220123292 sec. for 16 batches.
Forward pass needed 0.19199423980712887 out of 0.22427030277252197 sec. for 16 batches.
Forward pass needed 0.1913643503189087 out of 0.2247454414367676 sec. for 16 batches.
Forward pass needed 0.19221068668365482 out of 0.22442188835144045 sec. for 16 batches.
Forward pass needed 0.19364995193481446 out of 0.2267863044738769 sec. for 16 batches.
Forward pass needed 0.19259417629241937 out of 0.2246492147445679 sec. for 16 batches.
Forward pass needed 0.1904752330780029 out of 0.22599372768402098 sec. for 16 batches.
Forward pass needed 0.19192265605926515 out of 0.2244075508117676 sec. for 16 batches.
Forward pass needed 0.19205471992492676 out of 0.2242693452835083 sec. for 16 batches.
Forward 

In [106]:
reload_and_speed_test_model(gcn_model_ckpt_cs, data_cs, model_dir=model_dir1, device = 'cuda')

INFO:globals_MLP_projector: No inverse normalization for outputs is used.
INFO:levels_MLP_projector: No inverse normalization for outputs is used.
INFO:layers_MLP_projector: No inverse normalization for outputs is used.
INFO:GCN_Readout_MLP: No inverse normalization for outputs is used.


Forward pass needed 2.972647628784179 out of 2.9919323577880865 sec. for 16 batches.
Forward pass needed 2.8854375152587894 out of 2.9033256988525387 sec. for 16 batches.
Forward pass needed 2.884224716186523 out of 2.901990356445312 sec. for 16 batches.
Forward pass needed 2.884748825073242 out of 2.90565121459961 sec. for 16 batches.
Forward pass needed 2.884719268798828 out of 2.9026273193359375 sec. for 16 batches.
Forward pass needed 2.8858316345214847 out of 2.9038018798828125 sec. for 16 batches.
Forward pass needed 2.886290420532226 out of 2.904282043457031 sec. for 16 batches.
Forward pass needed 2.8837029724121095 out of 2.901633026123047 sec. for 16 batches.
Forward pass needed 2.890815261840821 out of 2.908788726806641 sec. for 16 batches.
Forward pass needed 2.8844036407470703 out of 2.9022075042724613 sec. for 16 batches.
Forward pass needed 2.885490646362305 out of 2.903402496337891 sec. for 16 batches.
Forward pass needed 2.897818984985351 out of 2.9158461608886714 sec.